In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
import json

file = json.load(open("json_extraction_dataset_500.json", "r"))
print(file[1])

In [ ]:
# Chekear GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

In [ ]:
from datasets import load_dataset

# 1. Definimos la plantilla del prompt estilo Alpaca
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# 2. Función para formatear los datos
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Insertamos los valores en la plantilla
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 3. Cargamos JSON
dataset = load_dataset("json", data_files="finanzas_dataset.jsonl", split="train")

# 4. Aplicamos el formato
dataset = dataset.map(formatting_prompts_func, batched = True)

# Verificación visual
print(dataset[0]["text"])

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        output_dir = "finanzas_lora_output" # Nombre de carpeta actualizado
    ),
)

In [ ]:
# Train the model
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("lora_model")

In [ ]:
FastLanguageModel.for_inference(model)

# Definimos una pregunta de prueba con el formato exacto del entrenamiento
instruction = "¿Cómo puedo empezar a invertir con poco dinero?"
input_context = "Tengo 22 años, estudiante, ahorro 50 euros al mes y riesgo bajo."

prompt = alpaca_prompt.format(instruction, input_context, "")

inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512, # Aumentamos tokens para que dé una respuesta completa
    use_cache = True
)

# Decodificamos la respuesta
response = tokenizer.batch_decode(outputs)[0]
print(response)